In [1]:
import codecs
import glob
import multiprocessing
import re
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import gensim
from keras.models import Sequential
from keras.layers.recurrent import LSTM, SimpleRNN
from sklearn.model_selection import train_test_split
from keras import optimizers

C:\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# PARAMETERS

In [2]:
w2vmodel="./model/chat_model1-300.w2v"   #filename
num_features = 300    #number of features in the model
maxsent = 10          #maximum words in a sentence.

### LOAD VECTOR MODEL 

In [444]:
tex2vec =  gensim.models.KeyedVectors.load(w2vmodel)

In [6]:
from chat_wordvec import *

### Cek data anda

In [8]:
aaf = swv_ar(twl("Saya tidak peduli. Itu terjadi menjadi nama saya.", rempunct=True, flat=True),frontpad=False)

C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [9]:
aaf.shape

(10, 300)

In [10]:
aaf

array([[ 0.20743306, -0.05019129, -0.00779339, ...,  0.06947476,
        -0.18152079, -0.12688521],
       [-0.11809482, -0.15485066, -0.01752153, ..., -0.06498069,
         0.19722947,  0.03699757],
       [-0.32420608, -0.3822366 , -0.15111044, ..., -0.17072281,
         0.00909948,  0.22414774],
       ...,
       [ 0.20743306, -0.05019129, -0.00779339, ...,  0.06947476,
        -0.18152079, -0.12688521],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ]])

### LOAD DATA ANDA

In [11]:
fileq = '../data/terjemah/test.dec-id.txt'
filea = '../data/terjemah/test.enc-id.txt'

In [12]:
context=read_ar(fileq,start=100, lines=100)
answer=read_ar(filea, start=100, lines=100)

Sentence overflow: ['karena', 'kita', 'punya', 'hal', 'yang', 'lebih', 'penting', 'untuk', 'dilakukan', 'seperti', 'mencari', 'tahu', 'siapa', 'yang', 'melakukannya']
Sentence overflow: ['apa', 'yang', 'harus', 'saya', 'lakukan', 'orang', 'itu', 'mencoba', 'merampokku', 'saya', 'akan', 'mengirim', 'polisi', 'kembali', 'saya', 'hanya', 'lupa']
Problem at phrase: leon
Sentence overflow: ['mungkin', 'anda', 'juga', 'tidak', 'peduli', 'tentang', 'itu', 'tersangka', 'tercantik', 'yang', 'saya', 'miliki', 'dalam', 'beberapa', 'waktu']
Problem at phrase: rumania
Problem at phrase: hongaria
Sentence overflow: ['saya', 'sedang', 'dalam', 'perjalanan', 'bersama', 'eddie', 'ludwig', 'dan', 'madison', 'para', 'tersangka', 'mungkin', 'sudah', 'ada', 'di', 'sana']
Problem at phrase: ludwig
Problem at phrase: madison
Sentence overflow: ['anda', 'adalah', 'juruselamatnya', 'apakah', 'dia', 'anak', 'yang', 'akan', 'kau', 'selamatkan', 'dari', 'gedung', 'yang', 'terbakar']
Problem at phrase: juruselamat

C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [13]:
print(context.shape)
print(answer.shape)

(100, 10, 300)
(100, 10, 300)


In [14]:
filen = '../data/textdata/dialog_kecil.txt'

In [15]:
with open(filen,"r",encoding="utf8") as f:
    fl = f.readlines()
    

In [16]:
Q=[]
A=[]
for i in range(len(fl)):
   if fl[i][0]=="A": 
        Q.append(fl[i][3:-1])
   if fl[i][0]=="B": 
        A.append(fl[i][3:-1])
    
A=A[0:len(Q)]

In [17]:
FQ=[]
FA=[]
for i in range(len(Q)):
    FQ.append(swv_ar(twl(Q[i],rempunct=True,flat=True),frontpad=False))
    FA.append(swv_ar(twl(A[i],rempunct=True,flat=True),frontpad=False))

C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Problem at phrase: middle
Problem at phrase: initial
Problem at phrase: dd
Problem at phrase: yy
Problem at phrase: januari
Sentence overflow: ['oh', 'ya', 'saya', 'membawanya', 'ke', 'sana', 'kemarin', 'dia', 'menunjukkan', 'satu', 'yang', 'sangat', 'dia', 'sukai']
Problem at phrase: ac
Problem at phrase: atm
Sentence overflow: ['yah', 'itu', 'memberi', 'saya', 'uang', 'tapi', 'itu', 'uang', 'saya', 'sendiri']
Sentence overflow: ['oh', 'apa', 'bagusnya', 'itu', 'saya', 'pikir', 'itu', 'memberi', 'anda', 'uang', 'gratis']
Sentence overflow: ['biasanya', 'tiba', 'di', 'sini', 'pada', 'siang', 'hari', 'pada', 'hari', 'selasa', 'yaitu', 'besok']
Sentence overflow: ['anda', 'akan', 'ingat', 'untuk', 'melakukannya', 'tetapi', 'anda', 'tidak', 'akan', 'punya', 'waktu', 'untuk', 'melakukannya']
Sentence overflow: ['tidak', 'saya', 'tidak', 'memerlukan', 'konverter', 'karena', 'saya', 'membeli', 'tv', 'digital']
Sentence overflow: ['oh', 'ya', 'saya', 'mendapat', 'enam', 'saluran', 'berbahasa'

Sentence overflow: ['ketika', 'dia', 'menjilati', 'bibirnya', 'kamu', 'tahu', 'dia', 'memiliki', 'tangan', 'yang', 'baik']
Sentence overflow: ['dia', 'tahu', 'kamu', 'tidak', 'bisa', 'membaca', 'pikirannya', 'mungkin', 'dia', 'mengira', 'kau', 'curang']
Sentence overflow: ['saya', 'akan', 'memiliki', 'radio', 'di', 'setiap', 'kamar', 'di', 'rumah', 'saya']
Sentence overflow: ['siapa', 'yang', 'tahu', 'mereka', 'mengatakan', 'akan', 'memperbaikinya', 'sebelum', 'musim', 'depan']
Sentence overflow: ['saya', 'ingin', 'tahu', 'apakah', 'saya', 'harus', 'membawa', 'sarung', 'tangan', 'saya']
Sentence overflow: ['lihat', 'dia', 'baru', 'saja', 'keluar', 'dari', 'mobil', 'dan', 'mulai', 'berlari']
Sentence overflow: ['kemarin', 'mereka', 'memberi', 'tahu', 'kami', 'tentang', 'seekor', 'anjing', 'dalam', 'pipa']
Sentence overflow: ['bulan', 'lalu', 'mereka', 'memberi', 'tahu', 'kami', 'tentang', 'seekor', 'tikus', 'di', 'sebuah', 'restoran']
Problem at phrase: beatles
Sentence overflow: ['mere

Sentence overflow: ['yah', 'itu', 'satu', 'jam', 'untuk', 'sampai', 'di', 'sana', 'jika', 'tidak', 'ada', 'masalah', 'lalu', 'lintas']
Sentence overflow: ['anda', 'tidak', 'pernah', 'tahu', 'apa', 'yang', 'mungkin', 'terjadi', 'di', 'jalan', 'raya', 'ini']
Problem at phrase: hawaii
Problem at phrase: oktober
Problem at phrase: oktober
Sentence overflow: ['baik', 'panggil', 'saja', 'meja', 'depan', 'mereka', 'akan', 'memberi', 'kita', 'lembaran', 'baru']
Problem at phrase: atlantik
Problem at phrase: hawaii
Sentence overflow: ['tidak', 'saya', 'tidak', 'ini', 'bulan', 'maret', 'waktu', 'hampir', 'habis', 'kursi', 'terjual', 'habis', 'sekarang']
Problem at phrase: maret
Sentence overflow: ['saya', 'yakin', 'ini', 'adalah', 'pertama', 'kalinya', 'anda', 'terbang', 'selama', 'natal']
Sentence overflow: ['harga', 'lebih', 'murah', 'tidak', 'akan', 'ada', 'gunanya', 'bagi', 'anda', 'jika', 'tidak', 'ada', 'kursi']
Sentence overflow: ['setiap', 'kali', 'kita', 'mendarat', 'atau', 'lepas', 'la

Sentence overflow: ['ya', 'yang', 'akan', 'meningkatkan', 'sekolah', 'kami', 'jalan', 'dan', 'rumah', 'sakit']
Problem at phrase: demokrat
Sentence overflow: ['saya', 'suka', 'yang', 'mana', 'petugas', 'pemadam', 'kebakaran', 'memberi', 'tahu', 'kami', 'mengapa', 'kami', 'harus', 'memilih', 'ya']
Problem at phrase: measure
Problem at phrase: schools
Problem at phrase: cost
Sentence overflow: ['yah', 'dia', 'tidak', 'bisa', 'menang', 'kecuali', 'kamu', 'dan', 'yang', 'lain', 'memilih', 'dia']
Sentence overflow: ['tetapi', 'bush', 'mengunjungi', 'beberapa', 'dari', 'mereka', 'di', 'rumah', 'sakit', 'sekali']
Problem at phrase: bush
Sentence overflow: ['dia', 'berbicara', 'kepada', 'mereka', 'dan', 'membuat', 'mereka', 'merasa', 'lebih', 'baik']
Sentence overflow: ['tidak', 'dia', 'terlalu', 'sibuk', 'menulis', 'buku', 'tentang', 'betapa', 'sulitnya', 'menjadi', 'presiden']
Sentence overflow: ['sama', 'seperti', 'sebelumnya', 'melalui', 'surat', 'semua', 'biaya', 'saya', 'adalah', 'cap', 

In [18]:
RQ=np.array(FQ)
RA=np.array(FA)

In [19]:
print(RQ.shape)
print(RA.shape)

(1307, 10, 300)
(1307, 10, 300)


In [20]:
Q[300]

'Biarkan saya mendapatkan kunci.'

### SIAPKAN DATA

In [21]:
x_train, x_test, y_train, y_test = train_test_split(RQ, RA, test_size=0.2, random_state=1)

In [22]:
    
model=Sequential()
model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

### LOADING MODEL SUDAH DI TRAINING

In [473]:
import keras
#model = keras.models.load_model("model-ep100.mdl")

# TRAINING MODEL

In [ ]:
model.fit(RQ,RA, epochs=1000)
model.save("model-ep2-100.mdl")

Epoch 1/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7491 - acc: 0.2698
Epoch 2/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7491 - acc: 0.2621
Epoch 3/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7494 - acc: 0.2653
Epoch 4/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7496 - acc: 0.2683
Epoch 5/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7499 - acc: 0.2718
Epoch 6/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7502 - acc: 0.2715
Epoch 7/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7503 - acc: 0.2748
Epoch 8/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7504 - acc: 0.2702
Epoch 9/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7490 - acc: 0.2648
Epoch 10/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7494 - acc: 0.2615

1307/1307 [==============================] - 6s 5ms/step - loss: -0.7569 - acc: 0.2972
Epoch 163/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7568 - acc: 0.2986
Epoch 164/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7567 - acc: 0.2980
Epoch 165/1000
1307/1307 [==============================] - 7s 5ms/step - loss: -0.7564 - acc: 0.2963
Epoch 166/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7562 - acc: 0.2966
Epoch 167/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7563 - acc: 0.2945
Epoch 168/1000
1307/1307 [==============================] - 7s 5ms/step - loss: -0.7563 - acc: 0.2947
Epoch 169/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7567 - acc: 0.2982
Epoch 170/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7566 - acc: 0.2965
Epoch 171/1000
1307/1307 [==============================] - 6s 5ms/step - loss: -0.7567 - acc: 0.

### PREDIKSI

In [841]:
import random
a = random.randint(0,1300)
print(Q[a])


def pred(sentence):
    predictions=model.predict(np.array([swv_ar(twl(sentence))]))
    result=decode(predictions)
    print(result)
    return result

#[tex2vec.most_similar([predictions[0][i]])[0] for i in range(maxsent)]

def rem_list(input):
    remlist=['LePastee','Tertawalah','Campari','Gundik','mastektomi']
    for i in remlist:
        try:
            ind=input.index(i)
            input.pop(ind)
        except:
            pass
    return input

def decode(pred):
    result=[]
    for i in range(maxsent):
        res = tex2vec.similar_by_vector(pred[0][i])[0][0]
        result.append(res)
        result = rem_list(result)
        strj = ' '.join(result)
    return strj

pred(Q[a])

Tanpa selai kacang!
Kehidupanmu Brodski Moussette Gazzara Goebbels


C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).


'Kehidupanmu Brodski Moussette Gazzara Goebbels'

# *******************************************************************
# Keras model of the chatbot: 
# *******************************************************************
from keras.layers import Input, Embedding, LSTM, Dense, RepeatVector, Bidirectional, Dropout, merge
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
maxlen_input=50
sentence_embedding_size = 300
weights_file="w.h5"
word_embedding_size=300
dictionary_size=7000

ad = Adam(lr=0.00005) 

input_context = Input(shape=(maxlen_input,), dtype='int32', name='input_context')
input_answer = Input(shape=(maxlen_input,), dtype='int32', name='input_answer')
LSTM_encoder = LSTM(sentence_embedding_size, init= 'lecun_uniform')
LSTM_decoder = LSTM(sentence_embedding_size, init= 'lecun_uniform')
if os.path.isfile(weights_file):
    Shared_Embedding = Embedding(output_dim=word_embedding_size, input_dim=dictionary_size, input_length=maxlen_input)
else:
    Shared_Embedding = Embedding(output_dim=word_embedding_size, input_dim=dictionary_size, weights=[embedding_matrix], input_length=maxlen_input)
word_embedding_context = Shared_Embedding(input_context)
context_embedding = LSTM_encoder(word_embedding_context)

word_embedding_answer = Shared_Embedding(input_answer)
answer_embedding = LSTM_decoder(word_embedding_answer)

merge_layer = merge([context_embedding, answer_embedding], mode='concat', concat_axis=1)
out = Dense(dictionary_size//2, activation="relu")(merge_layer)
out = Dense(dictionary_size, activation="softmax")(out)

model = Model(input=[input_context, input_answer], output = [out])

model.compile(loss='categorical_crossentropy', optimizer=ad)



In [294]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_context (InputLayer)      (None, 50)           0                                            
__________________________________________________________________________________________________
input_answer (InputLayer)       (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 50, 300)      2100000     input_context[0][0]              
                                                                 input_answer[0][0]               
__________________________________________________________________________________________________
lstm_21 (LSTM)                  (None, 300)          721200      embedding_3[0][0]                
__________

In [296]:
model.fit([x_train, y_train], y_train, batch_size=100, epochs=1)

ValueError: Error when checking input: expected input_context to have 2 dimensions, but got array with shape (80, 50, 300)

In [279]:
GLOVE_DIR="./"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'),encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
embedding_matrix = np.zeros((dictionary_size, word_embedding_size))


Found 400000 word vectors.
